# 모델 저장과 복원

## 설정
### 1. 설치와 임포트

In [2]:
!pip install -q h5py pyyaml

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.1.0'

In [6]:
#예제 데이터셋 받기
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### 2. 모델 정의

In [7]:
# 간단한 Sequential 모델을 반환합니다
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


# 모델 객체를 만듭니다
model = create_model()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### 3. 훈련하는 동안 체크포인트 저장하기
- 훈련 중간과 마지막에 체크포인트를 자동으로 저장하도록 해 다시 훈련하지 않고 모델을 재사용하거나 훈련 과정이 중지된 경우 이어서 훈련을 진행할 수 있음
- `tf.keras.callbacks.ModelCheckpoint` callback method를 통해 여러가지 매개변수를 제공받음

#### 체크포인트 콜백 사용하기

In [9]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # 훈련 단계에 콜백을 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 576/1000 [================>.............] - ETA: 0s - loss: 1.4827 - accuracy: 0.5642
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 408us/sample - loss: 1.1613 - accuracy: 0.6670 - val_loss: 0.6790 - val_accuracy: 0.8070
Epoch 2/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.4307 - accuracy: 0.8823
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 119us/sample - loss: 0.4285 - accuracy: 0.8810 - val_loss: 0.5334 - val_accuracy: 0.8400
Epoch 3/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.2716 - accuracy: 0.9333
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 121us/sample - loss: 0.2780 - accuracy: 0.9290 - val_loss: 0.4806 - val_accuracy: 0.8430
Epoch 4/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.2082 - accuracy: 0.9479
Epoch

텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때마다 업데이트


In [10]:
!ls {checkpoint_dir}

checkpoint                  cp.ckpt.index
cp.ckpt.data-00000-of-00001


- 훈련하지 않은 새로운 모델을 만들어 볼 것임.
- 가중치만 복원할 땐 원본 모델과 동일한 구조로 모델을 만들어야 함.
- 다른 <u>객체</u>이지만 동일한 구조로 모델을 만들었으므로 가중치를 공유할 수 있음

In [11]:
model = create_model()

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 2.4015 - accuracy: 0.0670
훈련되지 않은 모델의 정확도:  6.70%


- 체크포인트에서 가중치를 로드하고 다시 평가

In [12]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.3935 - accuracy: 0.8800
복원된 모델의 정확도: 88.00%


#### 체크포인트 콜백 매개변수
- 체크포인트 이름을 고유하게 만들거나 체크포인트 주기를 조정할 수 있음
- 새로운 모델을 훈련하고 다섯번의 에포크마다 고유한 이름으로 체크포인트 저장

In [15]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 다섯 번째 에포크마다 가중치를 저장합니다
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [16]:
! ls {checkpoint_dir}

checkpoint                       cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001 cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index               cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001 cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index               cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001 cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index               cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001 cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index               cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001 cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index               cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [17]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

- 모델 초기화하고 최근 체크포인트를 로드하여 테스트

In [18]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4837 - accuracy: 0.8780
복원된 모델의 정확도: 87.80%


### 4. 체크포인트 파일
- 체크포인트가 담고 있는 것
  - 모델의 가중치를 포함하는 하나 이상의 샤드(shard)
  - 가중치가 어느 샤드에 저장되어 있는지를 나타내는 인덱스 파일


### 5. 수동으로 가중치 저장하기
 - `Model.save_weights`를 사용하여 수동으로 가중치 저장

In [20]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 가중치를 복원합니다
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4837 - accuracy: 0.8780
복원된 모델의 정확도: 87.80%


### 6. 모델 전체를 저장하기
- 전체 모델을 하나의 파일에 저장.
- 가중치, 모델 구성, 옵티마이저에 지정한 설정끼지 포함
- 두 가지 방법: <b>`HDF5`</b>, <b>`Saved Model`</b>

#### 1) HDF5 파일로 저장하기
- 저장된 모델을 하나의 이진파일처럼 다룰 수 있음

In [22]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 296us/sample - loss: 1.1812 - accuracy: 0.6670
Epoch 2/5
1000/1000 [==============================] - 0s 53us/sample - loss: 0.4100 - accuracy: 0.8850
Epoch 3/5
1000/1000 [==============================] - 0s 54us/sample - loss: 0.2692 - accuracy: 0.9350
Epoch 4/5
1000/1000 [==============================] - 0s 53us/sample - loss: 0.2011 - accuracy: 0.9450
Epoch 5/5
1000/1000 [==============================] - 0s 55us/sample - loss: 0.1524 - accuracy: 0.9720


- <b>이 저장된 파일로 모델을 다시 만들어 보고 정확도 확인하기</b>

In [24]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [25]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))


1000/1000 - 0s - loss: 0.4464 - accuracy: 0.8590
복원된 모델의 정확도: 85.90%


#### 2) saved_model 사용하기
- 실험적 메서드로 현재 지원 안함